In [3]:
import usb.core
import usb.util
import time

# Find your device
dev = usb.core.find(idVendor=0x359F, idProduct=0x3003)

# Check if the device was found
if dev is None:
    raise ValueError('Device not found')

# Print the device speed
speed_dict = {
    1: "Low Speed",  # USB 1.0
    2: "Full Speed", # USB 1.1
    3: "High Speed", # USB 2.0
    4: "Super Speed" # USB 3.0
}

print(f'Device speed: {speed_dict.get(dev.speed, "Unknown Speed")}')

# Set the configuration
dev.set_configuration()

# Get an endpoint instance
cfg = dev.get_active_configuration()
intf = cfg[(0, 0)]

# print(intf)

ep_out = usb.util.find_descriptor(
    intf,
    custom_match=lambda e: usb.util.endpoint_direction(e.bEndpointAddress) == usb.util.ENDPOINT_OUT and e.bEndpointAddress == 0x02
)

ep_in = usb.util.find_descriptor(
    intf,
    custom_match=lambda e: usb.util.endpoint_direction(e.bEndpointAddress) == usb.util.ENDPOINT_IN and e.bEndpointAddress == 0x82
)

if ep_out is None or ep_in is None:
    raise ValueError('Endpoint not found')

## START 
print('\n\n')
## REG 4
# Define control transfer parameters
bmRequestType = 0b01000000
bRequest = 1
wValue = 4
wIndex = 0
data = [0x00, 0x00, 0x00, 0x00]

data_hex = [hex(x) for x in data]
print('REG 4 Write data:', data_hex)

# Perform control transfer
response1 = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)

print('REG 4 Write Response:', response1)


# Write data to the device
## data_to_write = b'\x01'
## ep_out.write(data_to_write)
## 

# Read data back from the device
start_time = time.time()
data_read_all = []
while True:
    data_read = dev.read(ep_in.bEndpointAddress, ep_in.wMaxPacketSize)
    data_read_all.extend(data_read)
    if time.time() - start_time >= 2:
        break

# while data_read_all[0] != 0xaa:
#     data_read_all = data_read_all[1:]
for i in range(len(data_read_all[:256])//16):
    print(f'Data:', bytes(data_read_all[i*16:i*16+16]))

## STOP
print('\n\n')
## REG 4
# Define control transfer parameters
bmRequestType = 0b01000000
bRequest = 1
wValue = 4
wIndex = 0
data = [0x00, 0x00, 0x00, 0x00]

data_hex = [hex(x) for x in data]
print('REG 4 Write data:', data_hex)

# Perform control transfer
response1 = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)

print('REG 4 Write Response:', response1)

# Release the device resources
usb.util.dispose_resources(dev)

Device speed: Super Speed



REG 4 Write data: ['0x0', '0x0', '0x0', '0x0']
REG 4 Write Response: 4
Data: b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f'
Data: b'\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f'
Data: b' !"#$%&\'()*+,-./'
Data: b'0123456789:;<=>?'
Data: b'@ABCDEFGHIJKLMNO'
Data: b'PQRSTUVWXYZ[\\]^_'
Data: b'`abcdefghijklmno'
Data: b'pqrstuvwxyz{|}~\x7f'
Data: b'\x80\x81\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x8b\x8c\x8d\x8e\x8f'
Data: b'\x90\x91\x92\x93\x94\x95\x96\x97\x98\x99\x9a\x9b\x9c\x9d\x9e\x9f'
Data: b'\xa0\xa1\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xab\xac\xad\xae\xaf'
Data: b'\xb0\xb1\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xbb\xbc\xbd\xbe\xbf'
Data: b'\xc0\xc1\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xcb\xcc\xcd\xce\xcf'
Data: b'\xd0\xd1\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xdb\xdc\xdd\xde\xdf'
Data: b'\xe0\xe1\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xeb\xec\xed\xee\xef'
Data: b'\xf0\xf1\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xfb\xfc\xfd\xfe\xff'



RE

In [15]:
# data_read = dev.read(ep_in.bEndpointAddress, ep_in.wMaxPacketSize)
# print('Data read:', data_read.tobytes())

usb.util.dispose_resources(dev)